In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten, Input
from tensorflow.keras.utils import to_categorical
import joblib

# Folder paths for your 4 classes
folders = {
    "Abnormal Heartbeat": r"E:\FYP\Cardi 2\Cardiovascular-Detection-using-ECG-images\preprocessed_1d\AHB",
    "Myocardial Infarction": r"E:\FYP\Cardi 2\Cardiovascular-Detection-using-ECG-images\preprocessed_1d\MI",
    "Normal": r"E:\FYP\Cardi 2\Cardiovascular-Detection-using-ECG-images\preprocessed_1d\NORMAL",
    "History of MI": r"E:\FYP\Cardi 2\Cardiovascular-Detection-using-ECG-images\preprocessed_1d\PM"
}

# Load CSVs and combine
all_data = []
for label, folder in folders.items():
    for file in os.listdir(folder):
        if file.endswith(".csv"):
            df = pd.read_csv(os.path.join(folder, file))
            df['Class'] = label
            all_data.append(df)

data = pd.concat(all_data, ignore_index=True)

# Features and labels
X = data.iloc[:, :255].values.astype(np.float32)  # Ensure numeric
y = data['Class'].values

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_onehot = to_categorical(y_encoded)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_onehot, test_size=0.2, random_state=42, stratify=y_onehot
)

# Reshape for CNN-LSTM: (samples, timesteps, features=1)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print(f"Training shape: {X_train.shape}, Test shape: {X_test.shape}")

In [ ]:
model = Sequential([
    Input(shape=(X_train.shape[1], 1)),

    # CNN block
    Conv1D(64, kernel_size=5, activation='relu', padding='same'),
    MaxPooling1D(pool_size=2),
    Conv1D(128, kernel_size=5, activation='relu', padding='same'),
    MaxPooling1D(pool_size=2),

    # LSTM block
    LSTM(64, return_sequences=False),

    # Fully connected
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(y_onehot.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32,
    verbose=1
)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Predictions
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# Classification report
print("=== Classification Report ===")
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))